# Creador de templates FIR para VHDL sin truncamiento a la salida
Para utilizar:

1-  Cambiar los parametros 

2-  Correr todas las celdas 

In [20]:
NUMBER_TAPS = 50
NUMBER_BITS_COEFS = 8
NUMBER_BITS_SAMPLES = 12
NUMBER_BITS_MULTIPLICATION = 20
# NUMBER BITS ADDITION INCREASE WITH EACH STAGE 
FILTER_NAME = 'FIR_TEMPLATE'

In [21]:
vhdl_architechture_header = '''
 ----------------------------------------------------------------
 --Module Architecture: FIR_IAN
 ----------------------------------------------------------------
 ARCHITECTURE rtl OF FIR_IAN IS
   -- Local Functions
   -- Type Definitions
   TYPE delay_pipeline_type IS ARRAY (NATURAL range <>) OF signed(11 DOWNTO 0); -- sfix16_En15
   -- Constants
   CONSTANT coeff1                         : signed(7 DOWNTO 0) := to_signed(-1, 8); -- sfix16_En16
'''
# COEFS
#lambda expression that returns a string with a template
vhdl_coef_template = lambda index, coef: f"CONSTANT coeff{index}                         : signed({NUMBER_BITS_COEFS-1} DOWNTO 0) := to_signed(REPLACE_ME, {NUMBER_BITS_COEFS});"
vhdl_coefs = [vhdl_coef_template(index, coef) for index, coef in enumerate(range(NUMBER_TAPS))]
#join vhdl_coefs a string separated by a new line character
vhdl_coefs = '\n'.join(vhdl_coefs) 

vhdl_delay_pipeline_type = f"TYPE delay_pipeline_type IS ARRAY (NATURAL range <>) OF signed({NUMBER_BITS_SAMPLES-1} DOWNTO 0); -- sfix16_En15"
vhdl_delay_pipeline_signal = f"signal delay_pipeline : delay_pipeline_type(0 to {NUMBER_TAPS-1});"
# Product template
#lambda expression that returns a string with a template
vhdl_signal_product_template = lambda index, coef: f"SIGNAL product{index}                         : signed({NUMBER_BITS_MULTIPLICATION-1} DOWNTO 0);"
vhdl_signal_product = [vhdl_signal_product_template(index, coef) for index, coef in enumerate(range(NUMBER_TAPS))]
vhdl_signal_product = '\n'.join(vhdl_signal_product) 

vhdl_signal_mul_temp_template = lambda index, coef: f"SIGNAL mul_temp{'' if index==0 else '_'+str(index)}                         : signed({NUMBER_BITS_MULTIPLICATION-1} DOWNTO 0);"
vhdl_signal_mul_temp = [vhdl_signal_mul_temp_template(index, coef) for index, coef in enumerate(range(NUMBER_TAPS))]
vhdl_signal_mul_temp = '\n'.join(vhdl_signal_mul_temp) 

# Sum template
#lambda expression that returns a string with a template
vhdl_signal_sum_template = lambda index, numbits: f"SIGNAL sum{index+1}                             : signed({numbits-1} DOWNTO 0);"
vhdl_signal_sum = [vhdl_signal_sum_template(index, NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1) for index in range(NUMBER_TAPS-1)]

vhdl_signal_add_template = lambda index, numbits: f"SIGNAL add_temp{'' if index==0 else '_'+str(index)}                             : signed({numbits-1} DOWNTO 0);"
vhdl_add_temp_sum = [vhdl_signal_add_template(index, NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1) for index in range(NUMBER_TAPS-1)]

vhdl_signal_sum = '\n'.join(vhdl_signal_sum)
vhdl_add_temp_sum = '\n'.join(vhdl_add_temp_sum)

vhdl_output_typeconvert = f"SIGNAL output_typeconvert               : signed({NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+NUMBER_TAPS-2} DOWNTO 0);"
vhdl_output_register = f"filter_out <= std_logic_vector(output_register);"

vhdl_delay_pipeline_process = '''
  -- Block Statements
  Delay_Pipeline_process : PROCESS (clk, reset)
  BEGIN
    IF reset = '1' THEN
      delay_pipeline(0 TO 9) <= (OTHERS => (OTHERS => '0'));
    ELSIF clk'event AND clk = '1' THEN
      IF clk_enable = '1' THEN
        delay_pipeline(0) <= signed(filter_in);
        delay_pipeline(1 TO 9) <= delay_pipeline(0 TO 8);
      END IF;
    END IF; 
  END PROCESS Delay_Pipeline_process;
  '''

# Multiplications
line_1_template = lambda index: f"mul_temp{'' if index==0 else '_'+str(index)} <= delay_pipeline({NUMBER_TAPS-1-index}) * coeff{NUMBER_TAPS-index}; \n"
line_2_template = lambda index: f"product{NUMBER_TAPS-1-index} <= resize(mul_temp{'' if index==0 else '_'+str(index)}, {NUMBER_BITS_MULTIPLICATION}); \n"

vhdl_multiplications_both_lines = [line_1_template(index) + line_2_template(index)   for index in range(NUMBER_TAPS)]
vhdl_multiplications_both_lines = "\n".join(vhdl_multiplications_both_lines)

# Sums
temp_lambda = lambda index: f"product{index}" if index == 0 else f"sum{index}" 
line_1_template = lambda index: f"add_temp{'' if index==0 else '_'+str(index)} <= resize({temp_lambda(index)}, {NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1}) + resize(product{index+1}, {NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1}); \n"
line_2_template = lambda index: f"sum{index+1} <= resize(add_temp{'' if index==0 else '_'+str(index)}, {NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+index+1}); \n"

vhdl_sums_both_lines = [line_1_template(index) + line_2_template(index)   for index in range(NUMBER_TAPS-1)]
vhdl_sums_both_lines = "\n".join(vhdl_sums_both_lines)



vhdl_output_process = '''
  Output_Register_process : PROCESS (clk, reset)
  BEGIN
    IF reset = '1' THEN
      output_register <= (OTHERS => '0');
    ELSIF clk'event AND clk = '1' THEN
      IF clk_enable = '1' THEN
        output_register <= output_typeconvert;
      END IF;
    END IF; 
  END PROCESS Output_Register_process;

    filter_out <= std_logic_vector(output_register);
'''    


In [24]:
import datetime
# Output a file with vhdl extension
with open(f'{FILTER_NAME}.vhd', 'w') as f:
    f.write(f'''
-- {FILTER_NAME}.vhd
-- {datetime.datetime.now()}
-- This filter has {NUMBER_TAPS} taps
-- The number of bits for the coefficients is {NUMBER_BITS_COEFS}
-- The number of bits for the samples is {NUMBER_BITS_SAMPLES}
-- The number of bits for the multiplications is {NUMBER_BITS_MULTIPLICATION}
-- The number of bits for the output is {NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+NUMBER_TAPS-2}


LIBRARY IEEE;
USE IEEE.std_logic_1164.all;
USE IEEE.numeric_std.ALL;

entity f'{FILTER_NAME} is
    Port ( clk : in std_logic;
           reset : in std_logic; 
           clk_enable : in std_logic;
           filter_in : in signed({NUMBER_BITS_SAMPLES-1} downto 0);
           filter_out : out signed({NUMBER_BITS_COEFS+NUMBER_BITS_SAMPLES+NUMBER_TAPS-2} downto 0));
end f'{FILTER_NAME};

architecture rtl of f'{FILTER_NAME} is

    -- Signals
    {vhdl_delay_pipeline_type}
    
    {vhdl_coefs}

    {vhdl_delay_pipeline_signal}
    
    {vhdl_signal_product}
    
    {vhdl_signal_mul_temp}
    
    {vhdl_signal_sum}
    
    {vhdl_add_temp_sum}
    
    {vhdl_output_typeconvert}
    
    {vhdl_output_register}

begin

    -- Block Statements
    {vhdl_delay_pipeline_process}

    -- Multiplications
    {vhdl_multiplications_both_lines}
    
    -- Sums
    {vhdl_sums_both_lines}

    -- Output
    output_typeconvert <= sum{NUMBER_TAPS-1}

    -- Output Process
    {vhdl_output_process}
end rtl;
''')


